In [3]:
import pandas as pd
import numpy as np

# Load your data
tle_df = pd.read_csv('data/starlink_tle.csv')
stations_df = pd.read_csv('data/stations_synthetic.csv')


In [4]:
stations_df

,station_id,station_name,latitude,longitude,elevation_m,antenna_gain_dBi
0,STAT001,"Wallops Island, VA",37.9375,-75.4667,6,45
1,STAT002,"Hawthorne, CA",33.9226,-118.3345,50,48
2,STAT003,"Madrid, Spain",40.4314,-3.7026,650,50
3,STAT004,"Seongnam, South Korea",37.4138,127.5183,80,47
4,STAT005,"Cape Town, South Africa",-33.9249,18.4241,15,46
5,STAT006,"Perth, Australia",-31.9505,115.8605,100,49


In [7]:
# Constants
earth_radius_km = 6371.0
light_speed_km_s = 299792.458

# Convert degrees to radians helper
def deg2rad(deg):
    return deg * (np.pi / 180.0)

# Function to convert (lat, lon, alt) to ECEF
def latlonalt_to_ecef(lat, lon, alt_km):
    lat_rad = deg2rad(lat)
    lon_rad = deg2rad(lon)
    r = earth_radius_km + alt_km
    x = r * np.cos(lat_rad) * np.cos(lon_rad)
    y = r * np.cos(lat_rad) * np.sin(lon_rad)
    z = r * np.sin(lat_rad)
    return x, y, z

# 1️⃣ Convert station lat/lon to ECEF (assuming alt = 0 km)
stations_df[['station_x', 'station_y', 'station_z']] = stations_df.apply(
    lambda row: pd.Series(latlonalt_to_ecef(row['latitude'], row['longitude'], 0)),
    axis=1
)

# 2️⃣ Convert satellite lat/lon/alt to ECEF
tle_df[['sat_x', 'sat_y', 'sat_z']] = tle_df.apply(
    lambda row: pd.Series(latlonalt_to_ecef(row['latitude'], row['longitude'], row['altitude_km'])),
    axis=1
)

# 3️⃣ Cartesian product: all satellites with all stations
merged = tle_df.assign(key=1).merge(stations_df.assign(key=1), on='key').drop('key', axis=1)

# 4️⃣ Calculate slant range (3D distance)
merged['slant_range_km'] = np.sqrt(
    (merged['sat_x'] - merged['station_x'])**2 +
    (merged['sat_y'] - merged['station_y'])**2 +
    (merged['sat_z'] - merged['station_z'])**2
)

# 5️⃣ Simulate synthetic network load (Poisson distributed)
np.random.seed(42)
merged['synthetic_msg_count'] = np.random.poisson(lam=50, size=len(merged))

# 6️⃣ Structured jitter (base jitter + 0.01 ms per message)
base_jitter_ms = 2.0
merged['structured_jitter_ms'] = base_jitter_ms + 0.01 * merged['synthetic_msg_count']

# 7️⃣ Calculate latency: (slant_range / c) + jitter
merged['latency_ms'] = (merged['slant_range_km'] / light_speed_km_s) * 1000 + merged['structured_jitter_ms']

# ✅ Optional: Check first few records
print(merged[['sat_id', 'station_id', 'slant_range_km', 'structured_jitter_ms', 'latency_ms']].head())


KeyError: 'latitude'